<a href="https://colab.research.google.com/github/secutron/FixMatch-pytorch/blob/master/FixMatch_pytorch_on_TPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FixMatch with Pytorch on TPUs - experimental

Code: https://github.com/vfdev-5/FixMatch-pytorch/


### Install requirements:

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'


try:
  import torch_xla
  import ignite
except ImportError:
  VERSION = "20210304"    
  # VERSION = "nightly"
  # VERSION = "20200607"
  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python pytorch-xla-env-setup.py --version $VERSION
  !pip install --upgrade git+https://github.com/pytorch/ignite
  !pip install --upgrade --pre hydra-core
  !pip install wandb
  !pip uninstall -y pillow && CC="cc -mavx2" pip install --no-cache-dir --force-reinstall pillow-simd


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  30819      0 --:--:-- --:--:-- --:--:-- 30819
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20210304 ...
Found existing installation: torch 1.9.0+cu102
     |████████████████████████████████| 57 kB 2.5 MB/s 
Uninstalling torch-1.9.0+cu102:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.272 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatibl

     |████████████████████████████████| 1.8 MB 4.8 MB/s 
     |████████████████████████████████| 138 kB 69.2 MB/s 
     |████████████████████████████████| 133 kB 63.5 MB/s 
     |████████████████████████████████| 170 kB 71.1 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 62 kB 729 kB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=ce4ab3904a651ebcfb1b08a696e7d7609aa8f030f60d7abfdbdcb5baec5bdc99
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=9acb1b30a4b7cd490ce90bbaff5b1e480ef6d3a5b472d2009b46949b896c0f01
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
  Attempting uninstall: urlli

### Download dataset:

In [ ]:
# Download dataset:
from torchvision.datasets import CIFAR10

CIFAR10("/tmp/cifar10", train=True, download=True);

Extracting /tmp/cifar10/cifar-10-python.tar.gz to /tmp/cifar10


### Get the code:

In [ ]:
!git clone https://github.com//FixMatch-pytorch/

#### Optionally, login to `W&B`

To skip logging to `W&B`, please set `online_exp_tracking.wandb=false` 

In [ ]:
# !wandb login <token>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


### Let's train ResNet18 model in a faster mode

In [ ]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python main_fixmatch.py distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true solver.num_epochs=50 \
    ssl.confidence_threshold=0.7 ema_decay=0.9 ssl.cta_update_every=15 solver.optimizer.params.lr=0.1

2020-06-15 14:45:26,154 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'
2020-06-15 14:45:26,154 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
2020-06-15 14:45:26,154 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f7b29cdbd90>' in 8 processes
2020-06-15 14:46:42,829 FixMatch Training INFO: name: fixmatch
seed: 543
debug: false
model: resnet18
num_classes: 10
ema_decay: 0.9
solver:
  unsupervised_criterion:
    cls: torch.nn.CrossEntropyLoss
    params:
      reduction: none
  num_epochs: 50
  epoch_length: 128
  checkpoint_every: 500
  validate_every: 1
  resume_from: null
  optimizer:
    cls: torch.optim.SGD
    params:
      lr: 0.1
      momentum: 0.9
      weight_decay: 0.0001
      nesterov: false
  supervised_criterion:
    cls: torch.nn.CrossEntropyLoss
  lr_scheduler:
    cls: torch.optim.lr_scheduler.CosineAnnealing

### Let's train WRN-28-2 model

In [ ]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python -u main_fixmatch.py model=WRN-28-2 distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true